## Introduction

dasdsfasf

In [25]:
import pandas as pd

data = pd.read_csv("loans_2007.csv", low_memory = False)
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


## 1. Data Cleaning

The dataset has already been cleaned up to remove some problematic or unnecessary columns, and remove columns with more than 50% of the values missing. However, we still need to look more closely through the columns for a few things:
* Whether the column leaks information about the future
* Whether the column is irrelevant to the borrower's ability to pay back the loan
* Whether a column needs to be reformatted
* Whether a column needs to be processed in order for useful information to be extracted from it
* Whether the column contains redundant information that is present elsewhere in the dataset

Of these, the first is the most important. If a column leaks information about the future, then if use it as a feature in our model for prediction we'll end up with an overfit model that won't be any good for predicting information on future loans (since we won't have that information yet).

We also need to select a column as a target column which we'll be trying to predict with our model.

To find more detailed information on the columns in our dataset, we can look at the data dictionary provided by Lending Club, found [here](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097).

In [26]:
list(data)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'pub_rec_bankruptcies',
 'tax_liens']

We'll start by cutting some columns. The following columns we'll cut because of data leakage: "funded_amnt", "funded_amnt_inv", "issue_d", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "last_credit_pull_d".

We'll cut the following because they're irrelevant, redundant, or hard to transform: "id", "member_id", "grade" (redundant, info in int_rate), "sub_grade" (ditto), "zip_code" (redundant, info in addr_state), "emp_title" (hard to transform).

While loan_status leaks information about the future, we need to make sure we keep it since it's going to serve as our target column. Our goal, after all, is to see which loan requests are worth taking because we expect the borrower to pay off the loan successfully.

In [27]:
data = data.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d", "zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "last_credit_pull_d"], axis = 1)
data.shape

(42538, 31)

There are a few columns we may need to transform, such as purpose, title, and addr_state, but we'll leave those for now.

### Cleaning the Target Column

Let's now look more closely at our target column.

In [28]:
data["loan_status"].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

Since we're trying to predict which loans will be successfully paid off, we're only interested in those loans which are either Full Paid or Charged Off. Those loans with are classified as Current, In Grace Period, Late (16 - 30 days), Late (31 -120 days), and Default are still undetermined.

We're also not interested in those loans which were fully paid or charged off but which no longer meet Lending Club's policy. Such loans will no longer get approved, and so if we include the data in those rows we'll be fitting our model to data which will be irrelevant in the future, because it relates to loans that will no longer meet the policy and we will therefore never need to evaluate.

This therefore turns our problem into one of binary classification. We're trying to predict whether a given loan will result in one of two values, either fully paid or charged off. We'll remove the rows for those loans whose statuses fall into neither category, and then we'll replace Fully Paid values with 1, and Charged Off with 0.

It should be noted that doing the above results in a class imbalance between the two outcomes. That is, the dataset we're training our model on contains far more loans which were fully paid than those that were charged off. This can result in the model having a bias towards predicting the class with more observations, in this case, full paid. This would be a disaster for us.

While we want to make as much money as possible by evaluating each prospective loan correctly, if the model were skewed towards predicting charged off rather than fully paid, this would be less of a concern. We'd miss out on funding some loans that would actually be good due to the bias of our model, but that would just mean less profit. However, given that our model would be biased towards predicting fully paid, we'd instead be funding loans which were actually bad, which would not just mean less profit, but potentially a loss.

As such, we'll need to remedy this issue, but we'll do so later. For now, we'll go ahead and alter our dataframe to contain only rows with loan statuses of fully paid or charged off.

In [31]:
data = data[(data['loan_status'] == "Fully Paid") | (data['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

data = data.replace(status_replace)

### Removing Useless Columns

Some columns may contain only one unique value. These columns will be useless for modelling as they contain no information to differentiate the loans from one another. To remove them, we first need to drop any null values from the columns, then find whether the remaining values are all alike. If they are, we'll add the column name to a list, and finally remove from the dataframe all columns in the list.

In [33]:
drop_columns = []

for col in list(data):
    non_null_unique = data[col].dropna().unique()
    if len(non_null_unique) == 1:
        drop_columns.append(col)

data = data.drop(drop_columns, axis = 1)

print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


After all of this, our dataframe is looking much smaller and easier to manage.

In [34]:
list(data)

['loan_amnt',
 'term',
 'int_rate',
 'installment',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'purpose',
 'title',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'pub_rec_bankruptcies']

## 2. Transforming Our Data

adfadf